In [5]:
import pandas as pd
import requests, random

r = requests.get("https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/stations.txt")
r.raise_for_status()
stations = r.text.strip().split("\n")
stations = random.sample(stations, k=10)
workload = random.choices(stations, k=100, weights=[0.3, 0.2] + [0.5/8]*8)


In [6]:
workload

['US1WISC0007',
 'US1WIWH0008',
 'US1WICR0006',
 'US1WISC0007',
 'US1WIWH0008',
 'US1WISC0007',
 'US1WISC0007',
 'US1WISB0026',
 'US1WIWH0008',
 'US1WISB0026',
 'US1WISC0007',
 'US1WISC0007',
 'US1WISC0007',
 'US1WIMT0009',
 'USC00475516',
 'US1WIWK0009',
 'US1WISC0007',
 'US1WIWH0008',
 'US1WIWK0009',
 'US1WISC0007',
 'US1WIMT0009',
 'US1WIWH0008',
 'US1WISC0007',
 'US1WISC0007',
 'US1WIWH0008',
 'US1WISC0007',
 'US1WIDA0025',
 'US1WIDA0025',
 'US1WICR0006',
 'US1WISC0007',
 'US1WISC0007',
 'US1WICR0006',
 'US1WISC0007',
 'US1WIWH0008',
 'US1WISC0007',
 'USC00475516',
 'US1WIWH0008',
 'US1WISC0007',
 'US1WICR0006',
 'US1WIWH0008',
 'US1WICR0006',
 'US1WIDA0025',
 'US1WIWH0008',
 'US1WIMT0009',
 'US1WISC0007',
 'US1WIMT0009',
 'US1WIWH0008',
 'US1WIWH0008',
 'US1WIWH0008',
 'US1WIMT0009',
 'US1WIMT0009',
 'US1WISC0007',
 'US1WISC0007',
 'US1WIMT0009',
 'US1WISB0026',
 'US1WISC0007',
 'US1WISC0007',
 'USC00475516',
 'US1WIWK0009',
 'US1WIMT0009',
 'US1WISC0007',
 'US1WIWH0008',
 'US1WIC

In [17]:
import time

cache = {} # key is the station, value in the DF
evict_order = []
cache_size = 4

#STATISTICS: HIT RATE AND AVERAGE LATENCY
hits = []
latencies = []

def get_df(station):
    
    start = time.time()
    if station in cache:
        ## LRU cache must increase prriority when somethine in the cache is used
        
        evict_order.remove(station)
        evict_order.append(station)
        ##print("hit")
        hits.append(1)
        return cache[station]
    else:
        ##print("miss")
        hits.append(0)
        df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
                         names=["station", "date", "element", "value", "m", "q", "s", "obs"])
        cache[station] = df
        evict_order.append(station)
        if(len(cache) > cache_size):
            ##print("evict")
            victim = evict_order.pop(0)
            cache.pop(victim)
            
    
        end = time.time()
        latencies.append((end - start) * 1000)
        
        return df


for station in workload[:100]:
    df = get_df(station)



print(sum(hits)/len(hits)) # hit rate
print(sum(latencies)/len(latencies)) # average latency
      

/tmp/ipykernel_35018/2531845661.py:25: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
/tmp/ipykernel_35018/2531845661.py:25: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
/tmp/ipykernel_35018/2531845661.py:25: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
/tmp/ipykernel_35018/2531845661.py:25: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"https://pages.cs.wisc.edu/~harter/cs639/data/wi-stations/{station}.csv.gz",
/tmp/ipykernel_35018/2531845661.py:25: DtypeWarning: Col

0.66
158.44350702622359


In [18]:
# tail latency: latency at a high peercentile

import numpy as np
np.quantile(latencies, .9) ## will get the 90th percentile latency
## will print the latency value where 90 percent are faster than the quanitle and 10 percent are slower.

460.20774841308594